In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/shared/3/cache/huggingface'
import json
import csv
import pandas as pd
import random, string, re, collections
import numpy as np
import torch
from argparse import ArgumentParser

from simpletransformers.classification import MultiLabelClassificationModel, MultiLabelClassificationArgs
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

2024-01-24 01:04:18.125411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
def split_labels(length,ratio=0.7,seed=0):
    '''
        generate a list of split labels (train, val) with the given split ratio and length
        input: 
            length: the size of the dataset
            ratio:training ratio
        return:
            agree_df: the processed annotation file for a single language 
    '''
    num_train = int(ratio * length)
    num_val = int(0.1 * length)
    num_test = length - num_train - num_val
# 7:1:2
    split_labels = ['train']*num_train + ['val']*num_val + ['test']*num_test
    random.seed(seed)
    random.shuffle(split_labels)
    return split_labels

# Multilabel classification

In [ ]:
def majority_accuracy(series):
    # If the majority of values are 1, return 1, otherwise return 0.
    return 1 if series.sum() > len(series) / 2 else 0

In [ ]:
exp_result_path = "path/to/exp_result.csv"
exp_result_df = pd.read_csv(exp_result_path)

In [ ]:
# save the idx of each role to keep track of the label-role pairs in multilabel classification 
role_idx = exp_result_df.groupby(['role']).count().index

role_idx_df = pd.DataFrame({
    'index': range(len(role_idx)),
    'role': llama_role_idx
}).reset_index(drop=True)

role_idx_df.to_csv("path/to/role_index.csv", index=False)

In [ ]:
result_grouped = exp_result_df.groupby(['role', 'question', 'dataset', 'full_question'])

role_accuracy_majority = result_grouped['accuracy'].agg(majority_accuracy).reset_index(name='majority_accuracy')
role_accuracy_majority.rename(columns={'majority_accuracy': 'accuracy'}, inplace=True)